In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import os


from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


# Loading the datasets

In [5]:
# non_merged pull request with the labels and complete characteristics for pr submitter
df_nonmerged = pd.read_csv("/Users/amirrshams/Library/CloudStorage/OneDrive-UniversityofWaterloo/Thesis/Dataset/Dataset/Non_Merged/Sample/temp/Sample_9000_pr.csv")
# merged pull request that has no labels as nothing is rejected
df_merged = pd.read_csv("/Users/amirrshams/Library/CloudStorage/OneDrive-UniversityofWaterloo/Thesis/Dataset/Dataset/Merged/pr_merged_final_April_2023.csv")
# all of the data that is merged and non merged
df_comp = pd.read_csv("/Users/amirrshams/Library/CloudStorage/OneDrive-UniversityofWaterloo/Thesis/Dataset/Dataset/pr_final_April_2023.csv")
# original dataset for getting the characterisitics of the pull request
df_TSE = pd.read_csv("/Users/amirrshams/Library/CloudStorage/OneDrive-UniversityofWaterloo/Thesis/Dataset/Reza's Dataset/TSE paper/2020-TSE-Developers-Perceptible-Ethnicity-and-PR-evaluation-main/Dataset/pull_requests.csv")

/var/folders/v7/m50ww3p142130jtt2scggl3r0000gq/T/ipykernel_4439/2698842826.py:6: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_comp = pd.read_csv("/Users/amirrshams/Library/CloudStorage/OneDrive-UniversityofWaterloo/Thesis/Dataset/Dataset/pr_final_April_2023.csv")
/var/folders/v7/m50ww3p142130jtt2scggl3r0000gq/T/ipykernel_4439/2698842826.py:8: DtypeWarning: Columns (8,14,31,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TSE = pd.read_csv("/Users/amirrshams/Library/CloudStorage/OneDrive-UniversityofWaterloo/Thesis/Dataset/Reza's Dataset/TSE paper/2020-TSE-Developers-Perceptible-Ethnicity-and-PR-evaluation-main/Dataset/pull_requests.csv")


In [4]:
df_nonmerged.head()

,Unnamed: 0,repo_id,status,pr_id,pullreq_id,api_url,url,pr_url,pr_api_url,author_id,author_desc_body,closer_id,comments_counts,comments,commit_counts,code_changes_counts,created_at,closed_at,author_country,closer_country,author_continent,same_country,author_eth,closer_eth,same_eth,prs_white,prs_api,prs_black,prs_hispanic,pri_white,pri_black,pri_api,pri_hispanic,prs_eth_8,prs_eth_7,prs_eth_9,prs_eth_diff,prs_eth_diff_2,prs_pri_same_nationality,prs_experience,prs_succ_rate,prs_popularity,prs_watched_repo,prs_followed_pri,prs_tenure_mnth,manual_analysis
0,2250,6,not-merged,2697422,4651,https://api.github.com/repos/cocos2d/cocos2d-x,https://github.com/cocos2d/cocos2d-x,https://github.com/cocos2d/cocos2d-x/pull/4651,https://api.github.com/repos/cocos2d/cocos2d-x...,CocosRobot,\N,NaN,0,[],1.0,1,2013-12-25T04:04:41Z,2013-12-25T05:41:54Z,\N,\N,\N,0,Unknown,API,0,0.889200,0.008001,0.088110,0.014689,0.059421,0.003402,0.933594,0.001673,Unknown,White,Unknown,White,White,0,371,67,4,0,0,28,No reason
1,5916,6,not-merged,7456135,11761,https://api.github.com/repos/cocos2d/cocos2d-x,https://github.com/cocos2d/cocos2d-x,https://github.com/cocos2d/cocos2d-x/pull/11761,https://api.github.com/repos/cocos2d/cocos2d-x...,CocosRobot,\N,NaN,0,[],1.0,9,2015-05-08T09:07:16Z,2015-05-11T01:53:15Z,\N,china,\N,0,Unknown,Unknown,0,0.889200,0.008001,0.088110,0.014689,0.569895,0.032570,0.367695,0.028239,Unknown,White,Unknown,White,White,0,1578,32,7,0,0,44,No reason
2,17935,183,not-merged,645270,2808,https://api.github.com/repos/zendframework/zen...,https://github.com/zendframework/zendframework,https://github.com/zendframework/zendframework...,https://api.github.com/repos/zendframework/zen...,blanchonvincent,Usage with static instead of self\n,NaN,0,[],1.0,1,2012-10-19T19:34:23Z,2012-10-30T19:46:43Z,france,united states,Europe,0,White,White,1,0.982306,0.003467,0.010741,0.002705,0.930962,0.031869,0.009878,0.025256,White,White,White,White,White,0,64,59,51,1,0,6,No reason
3,19620,183,not-merged,2203867,5219,https://api.github.com/repos/zendframework/zen...,https://github.com/zendframework/zendframework,https://github.com/zendframework/zendframework...,https://api.github.com/repos/zendframework/zen...,samsonasik,\N,NaN,3,"['Woah, ancient links spotting :O\n', '@Ocrami...",1.0,1,2013-10-03T19:34:04Z,2013-10-03T19:48:05Z,indonesia,spain,Asia,0,API,White,0,0.024666,0.975110,0.000109,0.000099,0.996368,0.002142,0.000852,0.000634,API,API,API,API,API,0,276,40,245,1,1,35,No reason
4,27952,340,not-merged,23622497,6647,https://api.github.com/repos/netty/netty,https://github.com/netty/netty,https://github.com/netty/netty/pull/6647,https://api.github.com/repos/netty/netty/pulls...,fenik17,Motivation:\r\n\r\n1. The use of the `Internet...,NaN,1,['Cherry-picked into 4.1 (970d310ec9ae3d0581ad...,1.0,3,2017-04-19T18:56:34Z,2017-04-20T03:23:50Z,russia,germany,Asia,0,White,White,1,0.990941,0.000334,0.000025,0.008697,0.992539,0.000581,0.003632,0.002701,White,White,White,White,White,0,11,0,1,1,1,8,Successful


#### Merging the characteristics of pr submitter with the complete dataset

In [10]:
df_TSE[df_TSE.duplicated(['pr_id'])].count()


Unnamed: 0                  0
repo_id                     0
pr_status                   0
prs_id                      0
pr_id                       0
repo_pr_tenure_mnth         0
repo_pr_popularity          0
repo_pr_team_size           0
perc_external_contribs      0
pr_opened_at                0
prs_country                 0
prs_continent               0
prm_country                 0
prc_country                 0
prs_pri_same_nationality    0
prs_experience              0
prs_succ_rate               0
pr_files_changed            0
pr_lines_changed            0
prs_main_team_member        0
prs_popularity              0
prs_watched_repo            0
prs_followed_pri            0
prs_tenure_mnth             0
pr_comments_counts          0
pr_num_commits              0
prs_eth                     0
prc_eth                     0
prm_eth                     0
pr_nth                      0
same_eth                    0
pri_id                      0
intra_branch                0
prm_id    

In [9]:
df_TSE = df_TSE.drop_duplicates(subset=['pr_id'], keep='first')


In [11]:
df_comp = pd.merge(df_comp, df_TSE[['pr_id', 'prs_pri_same_nationality', 'prs_experience', 'prs_succ_rate', 'prs_popularity', 'prs_watched_repo', 'prs_followed_pri', 'prs_tenure_mnth']], on='pr_id', how='left', validate='one_to_one')
df_merged = pd.merge(df_merged, df_TSE[['pr_id', 'prs_pri_same_nationality', 'prs_experience', 'prs_succ_rate', 'prs_popularity', 'prs_watched_repo', 'prs_followed_pri', 'prs_tenure_mnth']], on='pr_id', how='left', validate='one_to_one')

In [14]:
df_comp.head()

,Unnamed: 0,repo_id,status,pr_id,pullreq_id,api_url,url,pr_url,pr_api_url,author_id,author_desc_body,closer_id,comments_counts,comments,commit_counts,code_changes_counts,created_at,closed_at,author_country,closer_country,author_continent,same_country,author_eth,closer_eth,same_eth,prs_white,prs_api,prs_black,prs_hispanic,pri_white,pri_black,pri_api,pri_hispanic,prs_eth_8,prs_eth_7,prs_eth_9,prs_eth_diff,prs_eth_diff_2,prs_pri_same_nationality,prs_experience,prs_succ_rate,prs_popularity,prs_watched_repo,prs_followed_pri,prs_tenure_mnth
0,0,2,not-merged,33026,18,https://api.github.com/repos/kennethkalmer/ruo...,https://github.com/kennethkalmer/ruote-kit,https://github.com/kennethkalmer/ruote-kit/pul...,https://api.github.com/repos/kennethkalmer/ruo...,reedlaw,NaN,NaN,1,"[""Hello, thanks for the pull request. I fixed ...",1,1,2012-02-16T22:18:05Z,2012-02-18T02:13:05Z,united states,japan,North America,0,White,White,1,0.923822,0.030206,0.043507,0.001134,0.974386,0.011098,0.005310,0.008516,White,White,White,White,White,0,7,29,9,0,0,47
1,1,2,merged,852598,16,https://api.github.com/repos/kennethkalmer/ruo...,https://github.com/kennethkalmer/ruote-kit,https://github.com/kennethkalmer/ruote-kit/pul...,https://api.github.com/repos/kennethkalmer/ruo...,iamer,One more minor fix that helps with some crashe...,jmettraux,0,[],1,1,2012-01-04T10:23:49Z,2012-01-04T10:53:37Z,\N,japan,\N,0,Unknown,White,0,0.763503,0.187838,0.007003,0.001286,0.974386,0.011098,0.005310,0.008516,Unknown,White,Unknown,Unknown,Unknown,0,6,33,2,0,0,10
2,2,2,merged,1052366,14,https://api.github.com/repos/kennethkalmer/ruo...,https://github.com/kennethkalmer/ruote-kit,https://github.com/kennethkalmer/ruote-kit/pul...,https://api.github.com/repos/kennethkalmer/ruo...,iamer,Don't crash when a process has no root express...,jmettraux,0,[],1,1,2011-12-12T22:10:13Z,2011-12-12T22:13:44Z,\N,japan,\N,0,Unknown,White,0,0.763503,0.187838,0.007003,0.001286,0.974386,0.011098,0.005310,0.008516,Unknown,White,Unknown,Unknown,Unknown,0,5,20,2,0,0,9
3,3,3,merged,12308,64,https://api.github.com/repos/matplotlib/basemap,https://github.com/matplotlib/basemap,https://github.com/matplotlib/basemap/pull/64,https://api.github.com/repos/matplotlib/basema...,pmarshwx,Here's my first attempt to extend the new latl...,jswhit,7,"[""You've handled the map projection transforma...",3,1,2012-08-08T04:30:44Z,2012-08-20T22:32:28Z,united states,\N,North America,0,White,White,1,0.964081,0.001878,0.030626,0.001998,0.937730,0.058950,0.000949,0.001142,White,White,White,White,White,0,0,0,21,0,1,28
4,4,3,merged,148263,66,https://api.github.com/repos/matplotlib/basemap,https://github.com/matplotlib/basemap,https://github.com/matplotlib/basemap/pull/66,https://api.github.com/repos/matplotlib/basema...,pmarshwx,This fixes the scenario where plotting fails w...,jswhit,3,"[""I hadn't thought of that. It is a cleaner ap...",1,1,2012-08-19T20:21:14Z,2012-08-20T22:32:29Z,united states,\N,North America,0,White,White,1,0.964081,0.001878,0.030626,0.001998,0.937730,0.058950,0.000949,0.001142,White,White,White,White,White,0,2,100,21,0,1,28
